## NG production report with ppt

In [86]:
import pandas as pd
#file_path = r"D:\My Documents\Desktop\python_office\src\data\ng_data_january.xlsx"
file_path = r"D:\My Documents\Desktop\python_office\src\data\ng_data_febuary.xlsx"

month = file_path.split("\\")[-1].split(".")[0].split("_")[-1]
#print(month)
df = pd.read_excel(file_path)
df

febuary


,production_date,ng_topic,amount
0,2019-01-01,blur,1
1,2019-01-01,incomplete,1
2,2019-01-01,incomplete,1
3,2019-01-01,blur,1
4,2019-01-01,incomplete,1
...,...,...,...
9488,2019-01-31,short mold,1
9489,2019-01-31,no solder,1
9490,2019-01-31,incomplete,1
9491,2019-01-31,no solder,1


In [ ]:
pivot = pd.pivot_table(df,index="production_date",columns="ng_topic",values="amount",aggfunc="sum",margins=True,margins_name="Total")
pivot

In [ ]:
df_ng = pivot.loc[["Total"]].T[:-1]
df_ng = df_ng.sort_values(by="Total",ascending=False)
df_ng["cum_percentage"] = round(df_ng["Total"].cumsum()/df_ng["Total"].sum()*100,2)
df_ng

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.ticker import PercentFormatter
x = df_ng.index
y = df_ng["Total"]
y2 = df_ng["cum_percentage"]

fig,ax = plt.subplots(figsize=(22,10))
ax.bar(x,y)
ax.set_title(f"{month.upper()}'s NG PARETO CHART",fontsize = 40)
ax.set_xlabel("NG TOPIC",fontsize = 30)
ax.set_ylabel("FREQUENCY",fontsize = 30)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)

ax2 = ax.twinx()
ax2.plot(x,y2,color="red",marker="^",ms=10)
ax2.axhline(90,color="orange",linestyle="dashed")
ax2.yaxis.set_major_formatter(PercentFormatter())
ax2.set_ylabel("Cumulative Percentage",fontsize = 30)
ax2.yaxis.set_tick_params(labelsize=20)

#export
plt.savefig(r'export\pareto.png')

In [ ]:
df_time_serie = pivot.drop(columns=["Total"])[:-1]
df_time_serie

In [ ]:
#NG time series
fig,ax = plt.subplots(figsize=(22,10))
ax.plot(df_time_serie)
ax.set_title(f"{month.upper()}'s NG timeserie",fontsize = 40)
ax.set_xlabel("production date",fontsize = 30)
ax.tick_params(labelrotation=60)
ax.set_ylabel("FREQUENCY",fontsize = 30)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
ax.legend(list(df_time_serie.columns.values),fontsize = 13)
#export
plt.savefig(r'export\ng_timeserie.png')

In [ ]:
#Total NG
df_total_ng = pivot['Total'][:-1]
fig,ax = plt.subplots(figsize=(22,10))
ax.plot(df_total_ng)
ax.set_title(f"{month.upper()}'s TOTAL NG",fontsize = 40)
ax.set_xlabel("production date",fontsize = 30)
ax.tick_params(labelrotation=60)
ax.set_ylabel("FREQUENCY",fontsize = 30)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)

#export
plt.savefig(r'export\total_ng.png')

## Create report with template

In [ ]:
from pptx import Presentation
from pptx.dml.color import RGBColor
from pptx.util import Pt,Inches

prs = Presentation(r'D:\My Documents\Desktop\python_office\src\data\nmb_template.pptx')

# Create the title slide
title_slide_layout = prs.slide_layouts[0]
slide = prs.slides.add_slide(title_slide_layout)

# Add title
title = slide.shapes.title
title.text = "PRODUCTION MONTHLY REPORT"
title.text_frame.paragraphs[0].font.color.rgb = RGBColor(56, 48, 209)
title.text_frame.paragraphs[0].font.size = Pt(35)
title.text_frame.paragraphs[0].font.name = "Arial"
title.text_frame.paragraphs[0].font.bold = True
title.text_frame.paragraphs[0].font.underline = True

subtitle = slide.placeholders[1]
subtitle.text = "MIC division"
subtitle.text_frame.paragraphs[0].font.color.rgb = RGBColor(83, 76, 217)
subtitle.text_frame.paragraphs[0].font.size = Pt(20)
subtitle.text_frame.paragraphs[0].font.name = "Arial"


# add page 2
image_slide_layout = prs.slide_layouts[5] # add layout
slide = prs.slides.add_slide(image_slide_layout)

title = slide.shapes.title #title
title.text = f"{month.upper()}'s NG PARETO"

pareto = r'export\pareto.png' # add a picture
left = Inches(1.25)
top =  Inches(1.70)
pic = slide.shapes.add_picture(pareto,left,top,height=Inches(3.5))

# add page 3
image_slide_layout = prs.slide_layouts[5] # add layout
slide = prs.slides.add_slide(image_slide_layout)

title = slide.shapes.title #title
title.text = f"{month.upper()}'s NG TIME SERIES"

ng_time_serie = r'export\ng_timeserie.png' # add a picture
left = Inches(1.25)
top =  Inches(1.70)
pic = slide.shapes.add_picture(ng_time_serie,left,top,height=Inches(3.5))

# add page 4
image_slide_layout = prs.slide_layouts[5] # add layout
slide = prs.slides.add_slide(image_slide_layout)

title = slide.shapes.title #title
title.text = f"{month.upper()}'s NG TIME SERIES"

ng_time_serie = r'export\total_ng.png' # add a picture
left = Inches(1.25)
top =  Inches(1.70)
pic = slide.shapes.add_picture(ng_time_serie,left,top,height=Inches(3.5))

# Export
import datetime
now = datetime.datetime.now()
date_file_name = f'{str(now.date())}_{str(now.time()).split(".")[0].replace(":","_")}'

prs.save(f'export\monthly_report_with_template_{month}_{date_file_name}.pptx')